<a href="https://colab.research.google.com/github/olonok69/LLM_Notebooks/blob/main/langchain/use_cases/Langchain_Interacting_With_APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interacting with APIs

There are two primary ways to interface LLMs with external APIs:

- Functions: For example, OpenAI functions is one popular means of doing this.
- LLM-generated interface: Use an LLM with access to API documentation to create an interface.


klarna
# Langchian API docs

https://github.com/langchain-ai/langchain/tree/7fc07ba5df99b9fa8bef837b0fafa220bc5c932c/libs/langchain/langchain/chains/api



In [2]:
! pip install langchain langchain-openai  -q

In [15]:
import pprint

In [47]:
import os
from google.colab import userdata
import json
os.environ['OPENAI_API_KEY']=userdata.get('KEY_OPENAI')

Many APIs are already compatible with OpenAI function calling.

- Klarna  https://www.klarna.com/us/shopping/public/openai/v0/api-docs/
- Speak for translation   https://api.speak.com/openapi.yaml
- XKCD for comics https://gist.githubusercontent.com/roaldnefs/053e505b2b7a807290908fe9aa3e1f00/raw/0a212622ebfef501163f91e23803552411ed00e4/openapi.yaml


In [ ]:
 #Klarna https://www.klarna.com/us/shopping/public/openai/v0/api-docs/

In [53]:
from langchain.chains.openai_functions.openapi import get_openapi_chain

# chain = get_openapi_chain(
#     "https://www.klarna.com/us/shopping/public/openai/v0/api-docs/"
# )
# chain("What are some options for a men's large blue button down shirt")

# API Chain
We can also build our own interface to external APIs using the APIChain and provided API documentation.

In [8]:
import os
from google.colab import userdata
import json
os.environ['OPENAI_API_KEY']=userdata.get('KEY_OPENAI')

In [62]:
from langchain.chains import APIChain
from langchain.chains.api import open_meteo_docs
from langchain_openai import OpenAI

llm = OpenAI(temperature=0)
chain = APIChain.from_llm_and_api_docs(
    llm,
    open_meteo_docs.OPEN_METEO_DOCS,
    verbose=True,
    limit_to_domains=["https://api.open-meteo.com/"],
)
response = chain.run(
    "What is the weather like right now in London, UK in degrees Celsious?"
)



> Entering new APIChain chain...
 https://api.open-meteo.com/v1/forecast?latitude=51.5074&longitude=0.1278&hourly=temperature_2m&current_weather=true&temperature_unit=celsius&timezone=auto
{"latitude":51.5,"longitude":0.119999886,"generationtime_ms":0.09894371032714844,"utc_offset_seconds":0,"timezone":"Europe/London","timezone_abbreviation":"GMT","elevation":3.0,"current_weather_units":{"time":"iso8601","interval":"seconds","temperature":"°C","windspeed":"km/h","winddirection":"°","is_day":"","weathercode":"wmo code"},"current_weather":{"time":"2024-03-28T18:00","interval":900,"temperature":10.3,"windspeed":25.3,"winddirection":197,"is_day":1,"weathercode":3},"hourly_units":{"time":"iso8601","temperature_2m":"°C"},"hourly":{"time":["2024-03-28T00:00","2024-03-28T01:00","2024-03-28T02:00","2024-03-28T03:00","2024-03-28T04:00","2024-03-28T05:00","2024-03-28T06:00","2024-03-28T07:00","2024-03-28T08:00","2024-03-28T09:00","2024-03-28T10:00","2024-03-28T11:00","2024-03-28T12:00","2024-03

In [63]:
pprint.pprint(response)

(' The current temperature in London, UK is 10.3 degrees Celsius. This '
 'information was obtained from the API url: '
 'https://api.open-meteo.com/v1/forecast?latitude=51.5074&longitude=0.1278&hourly=temperature_2m&current_weather=true&temperature_unit=celsius&timezone=auto.')


In [16]:
pprint.pprint(open_meteo_docs.OPEN_METEO_DOCS)

('BASE URL: https://api.open-meteo.com/\n'
 '\n'
 'API Documentation\n'
 'The API endpoint /v1/forecast accepts a geographical coordinate, a list of '
 'weather variables and responds with a JSON hourly weather forecast for 7 '
 'days. Time always starts at 0:00 today and contains 168 hours. All URL '
 'parameters are listed below:\n'
 '\n'
 'Parameter\tFormat\tRequired\tDefault\tDescription\n'
 'latitude, longitude\tFloating point\tYes\t\tGeographical WGS84 coordinate of '
 'the location\n'
 'hourly\tString array\tNo\t\tA list of weather variables which should be '
 'returned. Values can be comma separated, or multiple &hourly= parameter in '
 'the URL can be used.\n'
 'daily\tString array\tNo\t\tA list of daily weather variable aggregations '
 'which should be returned. Values can be comma separated, or multiple &daily= '
 'parameter in the URL can be used. If daily weather variables are specified, '
 'parameter timezone is required.\n'
 'current_weather\tBool\tNo\tfalse\tInclude cur

# NEWSAPI

https://newsapi.org/

In [20]:
from langchain.chains.api import news_docs, podcast_docs

In [24]:
pprint.pprint(news_docs.NEWS_DOCS)

('API documentation:\n'
 'Endpoint: https://newsapi.org\n'
 'Top headlines /v2/top-headlines\n'
 '\n'
 'This endpoint provides live top and breaking headlines for a country, '
 'specific category in a country, single source, or multiple sources. You can '
 'also search with keywords. Articles are sorted by the earliest date '
 'published first.\n'
 '\n'
 'This endpoint is great for retrieving headlines for use with news tickers or '
 'similar.\n'
 'Request parameters\n'
 '\n'
 '    country | The 2-letter ISO 3166-1 code of the country you want to get '
 'headlines for. Possible options: ae ar at au be bg br ca ch cn co cu cz de '
 'eg fr gb gr hk hu id ie il in it jp kr lt lv ma mx my ng nl no nz ph pl pt '
 "ro rs ru sa se sg si sk th tr tw ua us ve za. Note: you can't mix this param "
 'with the sources param.\n'
 '    category | The category you want to get headlines for. Possible options: '
 'business entertainment general health science sports technology. Note: you '
 "can't mix t

In [66]:
os.environ["NEWSAPI_KEY"] = userdata.get('NEWSAPI_KEY')

headers = {"Authorization": f"Bearer {os.environ['NEWSAPI_KEY']}"}

In [68]:
chain = APIChain.from_llm_and_api_docs(
    llm,
    news_docs.NEWS_DOCS,
    verbose=True,
    headers=headers,
    limit_to_domains=["https://newsapi.org/"],
)
response = chain.run(
    "Latest top 5 business news  in us"
)



> Entering new APIChain chain...
 https://newsapi.org/v2/top-headlines?country=us&category=business&pageSize=5
{"status":"ok","totalResults":58,"articles":[{"source":{"id":null,"name":"Investor's Business Daily"},"author":"Investor's Business Daily","title":"Dow Jones Rises 250 Points As Nvidia, SMCI Fall; Tesla Gains Despite Roadblocks - Investor's Business Daily","description":"The Dow Jones and major indexes rebounded. Nvidia fell while Cathie Wood's Robinhood rose. Tesla got a downgrade on the stock market today.","url":"https://www.investors.com/market-trend/stock-market-today/dow-jones-sp500-nasdaq-nvidia-nvda-stock-tesla-cathie-wood/","urlToImage":"https://www.investors.com/wp-content/uploads/2018/07/stock-WallStreet-TrafficLight-01-shutter.jpg","publishedAt":"2024-03-27T18:37:10Z","content":"Major indexes kept the momentum going in the afternoon Wednesday as the Dow Jones Industrial Average and S&amp;P 500 looked to break three-day losing streaks. Nvidia and Super Micro Compu

In [69]:
pprint.pprint(response)

(' The API call returned the latest top 5 business news articles from the '
 'United States, including information on the Dow Jones and major indexes, the '
 'fight for AI talent, the Mega Millions jackpot, Visa and Mastercard fee '
 'settlement, and fast-food chains still using antibiotics in their meats.')


# Web requests

URL requests are such a common use-case that we have the LLMRequestsChain, which makes an HTTP GET request.


# Google search parameters

https://aicontentfy.com/en/blog/demystifying-google-search-url-parameters-and-how-to-use-them#:~:text=Google%20search%20URL%20parameters%20are,%2C%20location%2C%20or%20time%20frame.


In [30]:
from langchain.chains import LLMChain, LLMRequestsChain
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

In [31]:
template = """Between >>> and <<< are the raw search result text from google.
Extract the answer to the question '{query}' or say "not found" if the information is not contained.
Use the format
Extracted:<answer or "not found">
>>> {requests_result} <<<
Extracted:"""

PROMPT = PromptTemplate(
    input_variables=["query", "requests_result"],
    template=template,
)

In [70]:
chain = LLMRequestsChain(llm_chain=LLMChain(llm=OpenAI(temperature=0), prompt=PROMPT))
question = "What are the Three (3) cars most sold in Europe, and their respective sales volume?"
inputs = {
    "query": question,
    "url": "https://www.google.com/search?q=" + question.replace(" ", "+"),
}
response = chain.invoke(inputs)

In [71]:
response

{'query': 'What are the Three (3) cars most sold in Europe, and their respective sales volume?',
 'url': 'https://www.google.com/search?q=What+are+the+Three+(3)+cars+most+sold+in+Europe,+and+their+respective+sales+volume?',
 'output': 'Tesla Model Y 251,604 sales, +84% YoY. ... Dacia Sandero 234,715 sales, +18% YoY. ... Volkswagen T-Roc 204,610 sales, +13% YoY.'}

In [72]:
pprint.pprint(response['output'])

('Tesla Model Y 251,604 sales, +84% YoY. ... Dacia Sandero 234,715 sales, +18% '
 'YoY. ... Volkswagen T-Roc 204,610 sales, +13% YoY.')
